## ① データのアップデート

In [7]:
#ライブラリのインポート

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [8]:
#　requestsメソッドを使って、サイトからHTMLの情報を抽出する。
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTy1ZbSq5OAVbEqScDw_Uf_KfLKydg0uUWBaBEPK_DXaBPCHmwv4ru9MzZHHnVHVmRVvLvOlndJ2ZNZ/pubhtml?gid=159569114&amp;single=true&amp;widget=true&amp;headers=false&amp;gid=0&amp;range=A:F%22'
res = requests.get(url)

In [9]:
#　BeautifulSoup()の設定を行う。
soup = BeautifulSoup(res.text,'html.parser')

In [10]:
# 最新弾の情報を抽出する。
poke_new = soup.find('div',id ='1490875147')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_new.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,349):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)
    
# データの結合を行う。
df_new = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_new.columns = df_new.iloc[0]
df_new = df_new.drop(0)

#価格が欠損値になっている行を削除する。
df_new = df_new.dropna(subset = ['買取'])

#買取データの日付を取得する。
import datetime 
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_new.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [11]:
#### 通常の情報を抽出する。
poke_norm = soup.find('div',id ='159569114')
# s3(カード名、型番）の要素を抽出する。
card_data = poke_norm.find_all('td')

#情報をリストに格納する。
card_name = []
card_spec = []
card_rare = []
card_num = []
card_type = []
card_price = []

for i in range(0,3644):
    card_name.append(card_data[6*i].string)
    card_spec.append(card_data[6*i+1].string)
    card_rare.append(card_data[6*i+2].string)
    card_num.append(card_data[6*i+3].string)
    card_type.append(card_data[6*i+4].string)
    card_price.append(card_data[6*i+5].string)

# データの結合を行う。
df_norm = pd.concat([
    pd.DataFrame(card_name),
    pd.DataFrame(card_spec),
    pd.DataFrame(card_rare),
    pd.DataFrame(card_num),
    pd.DataFrame(card_type),
    pd.DataFrame(card_price)],axis = 1)
#0行目をカラムに設定する。
df_norm.columns = df_norm.iloc[0]
df_norm = df_norm.drop(0)

#価格が欠損値になっている行を削除する。
df_norm = df_norm.dropna(subset = ['買取価格'])

#買取データの日付を取得する。
import datetime
today = datetime.date.today()
today = today.strftime('%y-%m-%d')

df_norm.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [12]:
#更新日のデータを完成させる。
#データを結合する。
df_today = pd.concat([df_norm,df_new],ignore_index = True)
df_today.columns = ['カード名', '仕様', 'R', '型番', 'タイプ',today]

In [13]:
#集計データを読み込む。
df_preday = pd.read_csv('card_data.csv')

In [14]:
#結合のためのkeyを作る。(df_all)

df_today['key'] = df_today['カード名'].str.cat(df_today['型番'],sep = ' , ',na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['仕様'],sep = ' ,' ,na_rep='')
df_today['key'] = df_today['key'].str.cat(df_today['R'],sep = ' ,' ,na_rep='')

# print('df_todayのユニークなkeyの個数:',len(df_today['key'].unique()))
# print('df_todayのデータの列数:',len(df_today))
# print('df_todayのkeyに重複のあるデータ:\n',df_today[df_today['key'].duplicated()])

In [15]:
#結合のためのkeyを作る。(df_all)

df_preday['key'] = df_preday['カード名'].str.cat(df_preday['型番'],sep = ' , ',na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['仕様'],sep = ' ,' ,na_rep='')
df_preday['key'] = df_preday['key'].str.cat(df_preday['R'],sep = ' ,' ,na_rep='')

# print('df_predayのユニークなkeyの個数:',len(df_preday['key'].unique()))
# print('df_predayのデータの列数:',len(df_preday))
# print('df_predayのkeyに重複のあるデータ:\n',df_preday[df_preday['key'].duplicated()])

In [16]:
#keyが被る行にtodayの数値を代入する。

df_preday[today] = 0
df_preday

for i in range(len(df_preday)):
    for j in range(len(df_today)):
        if df_preday.loc[i,'key'] == df_today.loc[j,'key']:
            df_preday.loc[i,today] = df_today.loc[j,today]
            break

In [17]:
count = 0
for i in range(len(df_today)):
    flg = 0
    for j in range(len(df_preday)):
        if df_today.loc[i,'key'] == df_preday.loc[j,'key']:
            flg = 1
            break
            
    if flg == 0:
        df_preday = pd.concat([df_preday,df_today[i:i+1]])
        df_preday.reset_index(drop = True,inplace=True)
        count = count + 1

In [18]:
#データの整理を行う。
df_update = df_preday
df_update = df_update.drop('key',axis = 1)

#データの上書きを行う。
df_update.to_csv('card_data.csv',index = None)
df_update.to_csv('/Users/kanta/Desktop/Pokemon_work/data/{}.csv'.format(today),index = None)

## ② データの分析

In [19]:
#数値をint型に変換する。
df_update = df_update.replace(',','', regex=True)

#欠損値の処理を行う。
df_update.fillna({'カード名':'-','仕様':'-','型番':'-','タイプ':'-'},inplace = True)
df_update.fillna(0,inplace = True)
df_update = pd.concat([df_update.iloc[:,:4],df_update.iloc[:,5:].astype(int)],axis = 1)

#前日との値差を計算する。
from datetime import date, timedelta
today = datetime.date.today()
yesterday = today - timedelta(days = 1) 

today = today.strftime('%y-%m-%d')
yesterday = yesterday.strftime('%y-%m-%d')

df_update['前日との値差'] = df_update[today] - df_update[yesterday]  

In [20]:
#値上がりしたカードを抽出する。
print('■ 前日からの値上がりしたカード')
up_num = len(df_update[df_update['前日との値差'] > 0])
pd.set_option('display.max_rows',up_num)
df_update.sort_values('前日との値差',ascending=False)[0:up_num]

■ 前日からの値上がりしたカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,前日との値差
3254,リザードンVMAX＋リザードンV,HR仕様/SR仕様/当選通知書付き,P,103/S-P+104/S-P,730000,730000,730000,750000,20000
3680,ピカチュウ,封筒付き,P,001/S-P,0,0,0,16000,16000
1907,メガトウキョーのピカチュウ,-,P,204/XY-P,55000,55000,55000,65000,10000
3252,リザードンV,SR仕様,P,103/S-P,130000,130000,130000,140000,10000
1980,カナザワのピカチュウ,未開封,P,147/S-P,17000,17000,17000,22000,5000
2460,ニンフィアGX,-,HR,064/051,20000,20000,20000,25000,5000
1871,ピカチュウGX,-,P,393/SM-P,5000,5000,5000,10000,5000
3251,ミュウツーGX,SR仕様,P,363/SM-P,90000,90000,90000,95000,5000
1979,カナザワのピカチュウ,-,P,147/S-P,11000,11000,11000,15000,4000
1882,ボスごっこピカチュウ ギンガ団,未開封,P,194/SM-P,7500,7500,7500,11000,3500


In [21]:
print('■ 前日からの値下がりが大きいカード')
down_num = len(df_update[df_update['前日との値差'] < 0])
pd.set_option('display.max_rows',down_num)
df_update.sort_values('前日との値差',ascending=True).head(down_num)

■ 前日からの値下がりが大きいカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,前日との値差
3253,リザードンVMAX,HR仕様,P,104/S-P,620000,620000,620000,610000,-10000
3541,ミュウツーEX,-,SR,095/087,6000,6000,6000,0,-6000
3540,ミュウツーEX,フォトンウェーブ,SR,062/059,3000,3000,3000,0,-3000
3539,ミュウツーEX,バーストボール,SR,062/059,3000,3000,3000,0,-3000
3573,いちげきウーラオスVMAX,-,HR,084/070,3000,2800,2300,2000,-300
3571,いちげきウーラオスV,SA,SR,075/070,4000,4000,3700,3500,-200
946,グズマ＆ハラ,-,SR,105/095,1400,1400,1400,1200,-200
442,クワガノンV,-,SR,073/070,2000,2000,2000,1800,-200
3619,フヨウ,-,HR,087/070,2400,2400,2400,2200,-200
3570,いちげきウーラオスV,-,SR,074/070,1300,1300,1200,1000,-200


In [22]:
#新規に値段がついたカードを表示する。
print('■ 新規にデータを追記したカード')
df_update.tail(count)

■ 新規にデータを追記したカード


,カード名,仕様,R,型番,21-01-25,21-01-26,21-01-27,21-01-28,前日との値差
3678,ウルガモスGX,-,RR,013/095,0,0,0,10,10
3679,トゲピー＆ピィ＆ププリンGX,-,RR,094/173,0,0,0,10,10
3680,ピカチュウ,封筒付き,P,001/S-P,0,0,0,16000,16000
3681,オクタン,ミラー,-,025/131,0,0,0,30,30
3682,オクタン,ミラー,-,019/171,0,0,0,30,30
3683,ギラティナ◇,-,PR,058/173,0,0,0,30,30
3684,ギラティナ◇,-,PR,030/066,0,0,0,30,30
3685,チェリム,-,R,006/070,0,0,0,10,10
